In [1]:
# data['rate'] = data.gains_uvxy - data.gains_spy
# 对rate 进行正态判断，大于1个std，进行操作

In [2]:
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels
from statsmodels.tsa.stattools import coint
import pandas as pd
import scipy.stats as st
import os
import math

In [3]:
reload = True

file_name = 'uvxy_300d'

if not reload and  os.path.exists(file_name):
    hist = pd.read_csv(file_name)
else:
    tmp = yf.Ticker("UVXY")
    hist = tmp.history(period="300d")
    hist.to_csv(file_name)
    

file_name = 'spy_300d'
if not reload and  os.path.exists(file_name):
    spy_hist = pd.read_csv(file_name)
else:
    tmp = yf.Ticker("SPY")
    spy_hist = tmp.history(period="300d")
    spy_hist.to_csv(file_name)

print(hist.shape)
print(spy_hist.shape)

ConnectionError: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))

In [ ]:
hist.shape

In [ ]:
# hist.head(10)

In [ ]:
hist['gains'] = (hist.Close - hist.Close.shift(1))/hist.Close.shift(1) * 100
spy_hist['gains'] = (spy_hist.Close - spy_hist.Close.shift(1))/spy_hist.Close.shift(1) * 100

In [ ]:
hist['gains'].describe()

In [ ]:
spy_hist['gains'].describe()

In [ ]:
sns.distplot(hist.gains)

In [ ]:
# st.norm.cdf(-15, hist['gains'].mean(), hist['gains'].std())

In [ ]:
sns.distplot(spy_hist.gains)

In [ ]:
st.norm.cdf(-2, spy_hist['gains'].mean(), spy_hist['gains'].std())

In [ ]:
spy_hist.Close.plot(figsize=(10,10))
hist.Close.plot()

In [ ]:
spy_hist.gains.fillna(0, inplace=True)
hist.gains.fillna(0, inplace=True)

In [ ]:
# hist.gains.cumsum().plot()
# spy_hist.gains.cumsum().plot()

In [ ]:
# 差分
hist['close_diff']  = hist.Close - hist.Close.shift(1)  + 1000
spy_hist['close_diff']  = spy_hist.Close - spy_hist.Close.shift(1)  + 1000

hist.dropna(inplace=True)
spy_hist.dropna(inplace=True)

hist.head()

In [ ]:
# 对数
hist['close_log'] = hist['Close'].apply(math.log)
spy_hist['close_log'] = spy_hist['Close'].apply(math.log)

In [ ]:
spy_hist.head()

In [ ]:
score, pvalue, _ = coint(hist.gains, spy_hist.gains)
print(score, pvalue)

score, pvalue, _ = coint(hist.Close, spy_hist.Close)
print(score, pvalue)

score, pvalue, _ = coint(hist.close_diff, spy_hist.close_diff)
print(score, pvalue)

score, pvalue, _ = coint(hist.close_log, spy_hist.close_log)
print(score, pvalue)

In [ ]:
print(np.corrcoef(hist.Close, spy_hist.Close))
print(np.corrcoef(hist.close_diff, spy_hist.close_diff))
print(np.corrcoef(spy_hist.gains, hist.gains))

In [ ]:
hist.reset_index(inplace=True)
spy_hist.reset_index(inplace=True)

In [ ]:
col = ['Date','gains','Close','close_diff']
data = pd.merge(hist[col], spy_hist[col],suffixes=['_uvxy','_spy'], on='Date')
data.set_index(keys='Date')

In [ ]:
# data['rate'] = (data.gains_uvxy+1) / (data.gains_spy+1)
# data['rate'] = data.gains_uvxy - data.gains_spy          # 这个收益是 150
data['rate'] = data.close_diff_uvxy / data.close_diff_spy  # 差分+1000，这个收益是 188.78997135162348
data

In [ ]:
data.rate.describe()

In [ ]:
sns.distplot(data.rate)

In [ ]:
data.fillna(0,inplace=True)
data.isna().sum()

In [ ]:
print((data.rate>50).sum())
# data = data[abs(data['rate'])<50]
(data.rate>50).sum()

data.rate.plot()
plt.axhline(data.rate.mean(), color='red', linestyle='--') 

In [ ]:
def zscore(series):
    print(series.mean(), series.std())
    return (series - series.mean()) / np.std(series)

z_score = zscore(data.rate)
z_score.plot(figsize=(10,10))
plt.axhline(z_score.mean())
plt.axhline(1.0, color='red')
plt.axhline(-1.0, color='green')
plt.show()

# 开始预测

In [ ]:
print(z_score.shape)
train = z_score[:200]
test = z_score[200:]
print(train.shape, test.shape)

plt.figure(figsize=(15,7))
train.plot()

buy = train.copy()
sell = train.copy()

buy[train>-1] = 0
sell[train<1] = 0
# buy[~((data['gains_uvxy']>-1) & (data['gains_spy']>0))] = 0
# sell[~((data['gains_uvxy']<1) & (data['gains_spy']<0))] = 0



buy.plot(color='g', linestyle='None', marker='^')
sell.plot(color='r', linestyle='None', marker='^')
x1,x2,y1,y2 = plt.axis()
# plt.axis((x1,x2,data.rate.min(),data.rate.max()))
plt.legend(['Ratio', 'Buy Signal', 'Sell Signal'])
plt.show()

In [ ]:
# Plot the prices and buy and sell signals from z score
plt.figure(figsize=(10,10))
S1 = hist.iloc[:200].Close
S2 = spy_hist.iloc[:200].Close

S1.plot(color='b')
S2.plot(color='c')

buyR = 0*S1.copy()
sellR = 0*S1.copy()

# When buying the ratio, buy S1 and sell S2
buyR[buy!=0] = S1[buy!=0]
sellR[buy!=0] = S2[buy!=0]
# When selling the ratio, sell S1 and buy S2 
buyR[sell!=0] = S2[sell!=0]
sellR[sell!=0] = S1[sell!=0]

buyR.plot(color='g', linestyle='None', marker='^')
sellR.plot(color='r', linestyle='None', marker='^')

tmp = 150+ 10*z_score[0:200]
tmp.plot()

x1,x2,y1,y2 = plt.axis()
plt.axis((x1,x2,min(S1.min(),S2.min()),max(S1.max(),S2.max())))
plt.legend(['UVXY','SPY', 'Buy Signal', 'Sell Signal'])
plt.show()

In [ ]:
# Trade using a simple strategy
def trade(data_, z_score_, window1, window2):
    data_.reset_index(inplace=True,drop=True)
    z_score_.reset_index(inplace=True,drop=True)
    
    S1 = data_['gains_uvxy']
    S2 = data_['gains_spy']
    
    close = data_['Close_uvxy']
    date = data_['Date']
    # If window length is 0, algorithm doesn't make sense, so exit
    if (window1 == 0) or (window2 == 0):
        return 0
    
    # Compute rolling mean and rolling standard deviation
    
    # Simulate trading
    # Start with no money and no positions
    money = 0
    countS1 = 0
    countS2 = 0
    for i in range(len(S1)):
        # buy
        if z_score_[i]<-7:
          
            if countS1<0:
                money -= close[i]*abs(countS1)
                countS1 = 0
            
            money -= close[i]
            countS1 += 1
                
            print('Buying volatility %s at %f %f %f %s %s'%(date[i],z_score_[i], close[i], money+countS1*close[i], countS1,countS2))
        elif z_score_[i]>0:
            if countS1>0:
                money += close[i] * countS1
                countS1 = 0
            
            money += close[i]
            countS1 -= 1
                
            print('Selling volatility %s at %f %f %f %s %s'%(date[i], z_score_[i], close[i], money+countS1*close[i], countS1,countS2))
            
   
            
    return money+countS1*close.iloc[-1]

trade(data.iloc[:200],z_score.iloc[:200], 5, 60)

In [ ]:
data[hist['Date'] == "2020-04-01"]

In [ ]:
data[hist['Date'] == "2020-03-30"]

In [ ]:
trade(data.iloc[200:],z_score[200:], 5, 60)

In [ ]:
data.iloc[1]

In [ ]:
z_score.reset_index(inplace=True,drop=True)

In [ ]:
z_score